# Megoldás 1

In [1]:
from osgeo import ogr

# Shapefileok megnyitása
data_source = ogr.Open('shapefiles', 1)

# Épületek layer lekérése
epuletek_layer = data_source.GetLayerByName('epuletek')

# Ellenőrizzük, hogy létezik a layer, ha nem abbahagyjuk a futást
if not epuletek_layer:
    raise SystemExit("Shapefile cannot be opened!")
    
# Ellenőrizzük, hogy létezik-e az epuletek_bounding_dummy layer, ha igen töröljük
if data_source.GetLayer('epuletek_bounding_dummy'):
    data_source.DeleteLayer('epuletek_bounding_dummy')

# Az epuletek_bounding_dummy layer létrehozása
epuletek_bounding_layer = data_source.CreateLayer('epuletek_bounding_dummy', epuletek_layer.GetSpatialRef(), ogr.wkbPolygon)

# A layer definíciójának segítségével a feature létrehozása
epuletek_def = epuletek_bounding_layer.GetLayerDefn()
epuletek_bounding_feature = ogr.Feature(epuletek_def)

# Fő iteráció
for epuletek_feature in epuletek_layer:
    # Az adott feature gyűrűjének lekérdezése
    ring = epuletek_feature.geometry().GetGeometryRef(0)
    # A gyűrűből a min/max koordináták
    [min_x, max_x, min_y, max_y] = ring.GetEnvelope()
    # A bounding poligon és annak külső-gyűrűjének létrehozása
    bounding_ring = ogr.Geometry(ogr.wkbLinearRing)
    bounding_polygon = ogr.Geometry(ogr.wkbPolygon)
    # A gyűrűhöz hozzáadjuk a pontokat
    bounding_ring.AddPoint(min_x, min_y)
    bounding_ring.AddPoint(max_x, min_y)
    bounding_ring.AddPoint(max_x, max_y)
    bounding_ring.AddPoint(min_x, max_y)
    bounding_ring.CloseRings()
    # A poligonhoz hozzáadjuk a gyűrűt
    bounding_polygon.AddGeometry(bounding_ring)
    # Majd a feature-höz a poligont
    epuletek_bounding_feature.SetGeometry(bounding_polygon)
    # Illetve a layer-hez a feature-t
    epuletek_bounding_layer.CreateFeature(epuletek_bounding_feature)

# Bár a destructor syncel, azért nem ált manuálisan is meghívni
data_source.SyncToDisk()
del data_source

# Megoldás 2

In [2]:
from osgeo import ogr
from MinimumBoundingBox import MinimumBoundingBox, rectangle_corners

# Shapefileok megnyitása
data_source = ogr.Open('shapefiles', 1)

# Épületek layer lekérése
epuletek_layer = data_source.GetLayerByName('epuletek')

# Ellenőrizzük, hogy létezik a layer, ha nem abbahagyjuk a futást
if not epuletek_layer:
    raise SystemExit("Shapefile cannot be opened!")
    
# Ellenőrizzük, hogy létezik-e az epuletek_bounding layer, ha igen töröljük
if data_source.GetLayer('epuletek_bounding'):
    data_source.DeleteLayer('epuletek_bounding')

# Az epuletek_bounding layer létrehozása
epuletek_bounding_layer = data_source.CreateLayer('epuletek_bounding', epuletek_layer.GetSpatialRef(), ogr.wkbPolygon)

# A layer definíciójának segítségével a feature létrehozása
epuletek_def = epuletek_bounding_layer.GetLayerDefn()
epuletek_bounding_feature = ogr.Feature(epuletek_def)

# Fő iteráció
for epuletek_feature in epuletek_layer:
    # Az adott feature gyűrűjének lekérdezése
    ring = epuletek_feature.geometry().GetGeometryRef(0)
    # Lekérjük a gyűrű pontjait és kiszámítjuk a bounding boxot
    box = MinimumBoundingBox(ring.GetPoints())
    corners = rectangle_corners(box._asdict())
    # A bounding poligon és annak külső-gyűrűjének létrehozása
    bounding_ring = ogr.Geometry(ogr.wkbLinearRing)
    bounding_polygon = ogr.Geometry(ogr.wkbPolygon)
    # A gyűrűhöz hozzáadjuk a pontokat
    bounding_ring.AddPoint(corners[0][0], corners[0][1])
    bounding_ring.AddPoint(corners[1][0], corners[1][1])
    bounding_ring.AddPoint(corners[2][0], corners[2][1])
    bounding_ring.AddPoint(corners[3][0], corners[3][1])
    bounding_ring.CloseRings()
    # A poligonhoz hozzáadjuk a gyűrűt
    bounding_polygon.AddGeometry(bounding_ring)
    # Majd a feature-höz a poligont
    epuletek_bounding_feature.SetGeometry(bounding_polygon)
    # Illetve a layer-hez a feature-t
    epuletek_bounding_layer.CreateFeature(epuletek_bounding_feature)

# Bár a destructor syncel, azért nem ált manuálisan is meghívni
data_source.SyncToDisk()
del data_source